In [1]:
import sys
nb_out = sys.stdout

In [2]:
import tsaugmentation as tsag
sys.stdout = nb_out

Importing the tsaugmentation module. L. Roque. 
    Method to Test the Robustness of Hierarchical Time Series Forecasting Algorithms.



In [5]:
data = tsag.preprocessing.PreprocessDatasets('tourism').apply_preprocess()

Number of groups: 4
	state: 7
	zone: 27
	region: 76
	purpose: 4
Total number of series: 304
Number of points per series for train: 204
Total number of points: 228
Seasonality: 12
Forecast horizon: 24


In [6]:
train = data['train']['data']

In [7]:
train

array([[ 307, 2142,  202, ...,    3,    0,    5],
       [ 367,  609,  159, ...,    0,    0,    1],
       [ 543,  388,  100, ...,    0,    0,    2],
       ...,
       [ 589,  272,  174, ...,   15,    1,    0],
       [ 507,  269,  304, ...,    0,    1,    0],
       [ 768,  222,   72, ...,    2,    1,    0]])

Features for each node:
- Average number of visits at that node
- Number of connections (e.g. top node has 4 connections -> state, zone, region, purpose)

Only use train data to avoid leaking information into the train set. 

The adjencency matrix to build:

$$ \begin{matrix} 1 & 1 & 1& 1 \\
1 & 1 & 0 & 0 \\
0 & 0 & 1 & 1\\
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1\end{matrix}$$

In [33]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [ ]:
data['train']['groups_names']

{'state': array(['A', 'B', 'C', 'D', 'E', 'F', 'G'], dtype='<U1'),
 'zone': array(['AA', 'AB', 'AC', 'AD', 'AE', 'AF', 'BA', 'BB', 'BC', 'BD', 'BE',
        'CA', 'CB', 'CC', 'CD', 'DA', 'DB', 'DC', 'DD', 'EA', 'EB', 'EC',
        'FA', 'FB', 'FC', 'GA', 'GB'], dtype='<U2'),
 'region': array(['AAA', 'AAB', 'ABA', 'ABB', 'ACA', 'ADA', 'ADB', 'ADC', 'ADD',
        'AEA', 'AEB', 'AEC', 'AED', 'AFA', 'BAA', 'BAB', 'BAC', 'BBA',
        'BCA', 'BCB', 'BCC', 'BDA', 'BDB', 'BDC', 'BDD', 'BDE', 'BDF',
        'BEA', 'BEB', 'BEC', 'BED', 'BEE', 'BEF', 'BEG', 'BEH', 'CAA',
        'CAB', 'CAC', 'CBA', 'CBB', 'CBC', 'CBD', 'CCA', 'CCB', 'CCC',
        'CDA', 'CDB', 'DAA', 'DAB', 'DAC', 'DBA', 'DBB', 'DBC', 'DCA',
        'DCB', 'DCC', 'DCD', 'DDA', 'DDB', 'EAA', 'EAB', 'EAC', 'EBA',
        'ECA', 'FAA', 'FBA', 'FBB', 'FCA', 'FCB', 'GAA', 'GAB', 'GAC',
        'GBA', 'GBB', 'GBC', 'GBD'], dtype='<U3'),
 'purpose': array(['Bus', 'Hol', 'Oth', 'Vis'], dtype='<U3')}

In [ ]:
enc = OneHotEncoder(
    
)